In [1]:
#pip install pyodbc

In [2]:
#pip install openpyxl

In [3]:
#pip install sqlalchemy

In [4]:
# Emergency reload from github
#url = 'https://raw.githubusercontent.com/MarkMarquez0224/Resume-Portfolio/master/VC_News_Daily.csv'
# recovery_file = pd.read_csv(url)

In [5]:
#recovery_file.to_csv('VC_News_Daily.csv',index=False)

In [6]:
from bs4 import BeautifulSoup

import os

import pandas as pd
import numpy as np

import pyodbc

import requests

import sqlalchemy
from sqlalchemy import create_engine

import time

In [7]:
os.chdir('C:\\Users\\marky\\OneDrive\\Documents\\Data Science\\Venture Funding and Companies\\')
os.getcwd()

'C:\\Users\\marky\\OneDrive\\Documents\\Data Science\\Venture Funding and Companies'

In [8]:
os.listdir()

['.ipynb_checkpoints',
 'VC News Daily - Web Scrap and Store.pdf',
 'VC News Web Scrap and Store.ipynb',
 'VC News Web Scrap and Store.ipynb - V2.ipynb',
 'VC_News_Daily-Test.csv',
 'VC_News_Daily.csv',
 'VC_News_Daily.xlsx',
 'VC_News_Daily_5_6.csv',
 'VC_News_Daily_Clean.csv',
 'VC_News_Daily_Clean.xlsx']

In [9]:
# Read website into 'BeautifulSoup' to help parse the website
website_soup = BeautifulSoup(requests.get('https://vcnewsdaily.com/').text)

In [10]:
# Read funding activity and store into list
funding_activity = website_soup.find_all('div',class_='col-md-8')

In [11]:
# For loop to read in 'funding_activity', get information(date, headline, website link), and store into list
funding_activity_list = [[funding_activity[i].small.text,funding_activity[i].h5.text,funding_activity[i].a.get('href')] for i in range(len(funding_activity))]
df_funding_activity_list = pd.DataFrame(funding_activity_list,columns=['Funding Date','Funding Title','Website Article Link'])

In [12]:
df_funding_activity_list.head()

,Funding Date,Funding Title,Website Article Link
0,17 Oct 2020,Ateios Raises $1.25M in Seed Funding,https://vcnewsdaily.com/ateios/venture-capital...
1,17 Oct 2020,Vineti Grabs $33M Series C Extension,https://vcnewsdaily.com/vineti/venture-capital...
2,16 Oct 2020,Dren Bio Announces a $60M Series A,https://vcnewsdaily.com/dren-bio/venture-capit...
3,16 Oct 2020,Sanity Announces $9.3M in Series A,https://vcnewsdaily.com/sanity.io/venture-capi...
4,16 Oct 2020,Popmenu Raises $17M Series B,https://vcnewsdaily.com/popmenu/venture-capita...


In [13]:
df_funding_activity_list.shape

(30, 3)

In [14]:
# Articles from VC News Daily put into series
website_article_links = df_funding_activity_list['Website Article Link']

In [15]:
# Articles from our CSV download put into a series
website_article_links_total = pd.read_csv('VC_News_Daily_Clean.csv')['Website Article Link']

In [16]:
# Checking for nbumber of links that aren't already saved
website_article_links_length = len(website_article_links[~website_article_links_total.isin(website_article_links)])
website_article_links_length

0

In [17]:
df_funding_activity_list = df_funding_activity_list[0:website_article_links_length]
df_funding_activity_list.head(len(df_funding_activity_list))

,Funding Date,Funding Title,Website Article Link


In [18]:
df_funding_activity_list.dtypes.to_dict()

{'Funding Date': dtype('O'),
 'Funding Title': dtype('O'),
 'Website Article Link': dtype('O')}

In [19]:
funding_activity_funding=[]
if website_article_links_length > 0:
    for i in range(len(df_funding_activity_list)):
        company_text = BeautifulSoup(requests.get(website_article_links[i]).text)
        funding_headline = company_text.find_all(id='summary')[0].text.strip('\n').strip('\t')
        if len(funding_headline.split(', ')) > 1:
            company_city = funding_headline.title().split(', ')[0]
            company_state = funding_headline.upper().split(', ')[1]
        else:
            company_city = 'N/A'
            company_state = 'N/A'
        website_company_link = company_text.find_all(id='fullArticle')[0].a.get('href')
        funding_article = company_text.find_all(id='fullArticle')[1].text
        funding_activity_funding.append([funding_headline,company_city,company_state,website_company_link,funding_article])

df_funding_activity_funding = pd.DataFrame(funding_activity_funding,columns=['Funding Headline','City','State','Website Company Link','Funding Article'])
df_funding_activity_funding.head(len(df_funding_activity_list))

,Funding Headline,City,State,Website Company Link,Funding Article


In [20]:
df_funding_activity_funding.shape

(0, 5)

In [21]:
df_funding_activity_funding.head()

,Funding Headline,City,State,Website Company Link,Funding Article


In [22]:
website_company_links = df_funding_activity_funding['Website Company Link']
# website_company_links

In [23]:
# i

In [24]:
#debug_info = BeautifulSoup(requests.get(website_company_links[i]).text)

In [25]:
#debug_info.find_all('td') == []

In [26]:
# Debug code
#if debug_info.find_all('td') == True:
#    funding_type = debug_info.find_all('td')[-2].text.strip('\n')
#    funding_amount = debug_info.find_all('td')[-3].text.strip('\n')
#else:
#    funding_type = 'N/A'
#    funding_amount = 'N/A'    

# funding_amount

In [27]:
funding_company_list = []
if website_article_links_length > 0:
    for i in range(len(df_funding_activity_list)):
        company_info = BeautifulSoup(requests.get(website_company_links[i]).text)
        company_name = company_info.find(class_='col-md-7 d-flex align-items-center').h5.text
        company_desc = company_info.find(class_='mt-5').p.text
        if company_info.find_all('td') == []:
            funding_type = None
            funding_amount = None 
        else:
            funding_type = company_info.find_all('td')[-2].text.strip('\n')
            funding_amount = company_info.find_all('td')[-3].text.strip('\n')
            if funding_amount == 'undisclosed':
                funding_amount = None
        
        funding_company_list.append([company_name,company_desc,funding_type,funding_amount])


df_funding_company_list = pd.DataFrame(funding_company_list,columns=['Company','Company Description','Latest Funding Round','Latest Funding Amount'])
df_funding_company_list.head(len(df_funding_activity_list))

,Company,Company Description,Latest Funding Round,Latest Funding Amount


In [28]:
df_funding_company_list['Latest Funding Amount'].str.strip('$')\
        .replace(',', '', regex=True)

Series([], Name: Latest Funding Amount, dtype: object)

In [29]:
df_funding_company_list.head()

,Company,Company Description,Latest Funding Round,Latest Funding Amount


In [30]:
df_funding_company_list.shape

(0, 4)

In [31]:
df_funding_data = pd.concat([df_funding_company_list,df_funding_activity_list,df_funding_activity_funding], axis=1)
df_funding_data.head()

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article


In [32]:
df_funding_data.shape

(0, 12)

In [33]:
total_funding_data = pd.read_csv('VC_News_Daily_Clean.csv')

In [34]:
total_funding_data

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Ateios,Ateios is developing a new process to reshape ...,Seed,1250000,17 Oct 2020,Ateios Raises $1.25M in Seed Funding,https://vcnewsdaily.com/ateios/venture-capital...,"NEWBERRY, IN, Ateios announced closing an over...",Newberry,IN,https://vcnewsdaily.com/Ateios/venture-funding...,Ateios (pronounced 'Ah-TAY-ohs') has developed...
1,Vineti,Vineti is the first commercial cloud-based pla...,Series C Extension,33000000,17 Oct 2020,Vineti Grabs $33M Series C Extension,https://vcnewsdaily.com/vineti/venture-capital...,"SAN FRANCISCO, CA, Vineti announced the closin...",San Francisco,CA,https://vcnewsdaily.com/Vineti/venture-funding...,"Vineti, a leading digital platform of record f..."
2,Dren Bio,Dren Bio is a biotechnology company developing...,Series A,60000000,16 Oct 2020,Dren Bio Announces a $60M Series A,https://vcnewsdaily.com/dren-bio/venture-capit...,"SAN CARLOS, CA, Dren Bio announced the closing...",San Carlos,CA,https://vcnewsdaily.com/Dren%20Bio/venture-fun...,"Dren Bio, Inc., a company developing powerful ..."
3,Sanity.io,Sanity is a content platform that helps people...,Series A,9300000,16 Oct 2020,Sanity Announces $9.3M in Series A,https://vcnewsdaily.com/sanity.io/venture-capi...,"SAN FRANCISCO, CA, Sanity.io has raised $9.3M ...",San Francisco,CA,https://vcnewsdaily.com/Sanity.io/venture-fund...,"Sanity.io, the developer-focused content platf..."
4,Popmenu,Popmenu helps restaurants take real-time contr...,Series B,17000000,16 Oct 2020,Popmenu Raises $17M Series B,https://vcnewsdaily.com/popmenu/venture-capita...,"ATLANTA, GA, Popmenu has raised a $17 million ...",Atlanta,GA,https://vcnewsdaily.com/Popmenu/venture-fundin...,"Popmenu, an all-in-one digital marketing and o..."
...,...,...,...,...,...,...,...,...,...,...,...,...
812,Guilded,Guilded is the ultimate chat platform for gami...,Series A,7000000,30 Apr 2020,Guilded Pulls In $7M Series A Financing,https://vcnewsdaily.com/guilded/venture-capita...,"SAN FRANCISCO, CA, Guilded has raised $7 milli...",San Francisco,CA,https://vcnewsdaily.com/Guilded/venture-fundin...,"The funding was led by Matrix Partners, with p..."
813,Stash,Stash is the fast-growing investment app for m...,Series F,112000000,29 Apr 2020,Stash Raises $112M in Series F Funding,https://vcnewsdaily.com/stash/venture-capital-...,"NEW YORK, NY, Stash has raised a $112 million ...",New York,NY,https://vcnewsdaily.com/Stash/venture-funding.php,Stash has raised a $112 million Series F round...
814,Fat Snax,"Fat Snax has been providing low-carb, yet indu...",Series A,4500000,29 Apr 2020,Fat Snax Closes $4.5M Series A,https://vcnewsdaily.com/fat-snax/venture-capit...,"BROOKLYN, NY, Fat Snax has closed a $4.5M Seri...",Brooklyn,NY,https://vcnewsdaily.com/Fat%20Snax/venture-fun...,"Fat Snax, the leading low-carb & keto snack br..."
815,Wise,"Founded in December 2018, Wise, a Techstars NY...",Seed,5700000,29 Apr 2020,Wise Raises $5.7M Seed Round,https://vcnewsdaily.com/wise/venture-capital-f...,"SAN MATEO, CA, FinTech startup that offers ban...",San Mateo,CA,https://vcnewsdaily.com/Wise/venture-funding.php,"Wise, a FinTech startup that offers banking an..."


In [35]:
# total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].astype('int64')

In [36]:
df_funding_data.dtypes

Company                  object
Company Description      object
Latest Funding Round     object
Latest Funding Amount    object
Funding Date             object
Funding Title            object
Website Article Link     object
Funding Headline         object
City                     object
State                    object
Website Company Link     object
Funding Article          object
dtype: object

In [37]:
total_funding_data = pd.concat([df_funding_data,total_funding_data],axis=0).reset_index(drop=True)

In [38]:
# Temporary fix for duplicates (when needed) # total_funding_data = total_funding_data.iloc[11:].reset_index(drop=True)

In [39]:
# total_funding_data.iloc[11:].reset_index(drop=True)

In [40]:
total_funding_data.head(20)

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Ateios,Ateios is developing a new process to reshape ...,Seed,1250000,17 Oct 2020,Ateios Raises $1.25M in Seed Funding,https://vcnewsdaily.com/ateios/venture-capital...,"NEWBERRY, IN, Ateios announced closing an over...",Newberry,IN,https://vcnewsdaily.com/Ateios/venture-funding...,Ateios (pronounced 'Ah-TAY-ohs') has developed...
1,Vineti,Vineti is the first commercial cloud-based pla...,Series C Extension,33000000,17 Oct 2020,Vineti Grabs $33M Series C Extension,https://vcnewsdaily.com/vineti/venture-capital...,"SAN FRANCISCO, CA, Vineti announced the closin...",San Francisco,CA,https://vcnewsdaily.com/Vineti/venture-funding...,"Vineti, a leading digital platform of record f..."
2,Dren Bio,Dren Bio is a biotechnology company developing...,Series A,60000000,16 Oct 2020,Dren Bio Announces a $60M Series A,https://vcnewsdaily.com/dren-bio/venture-capit...,"SAN CARLOS, CA, Dren Bio announced the closing...",San Carlos,CA,https://vcnewsdaily.com/Dren%20Bio/venture-fun...,"Dren Bio, Inc., a company developing powerful ..."
3,Sanity.io,Sanity is a content platform that helps people...,Series A,9300000,16 Oct 2020,Sanity Announces $9.3M in Series A,https://vcnewsdaily.com/sanity.io/venture-capi...,"SAN FRANCISCO, CA, Sanity.io has raised $9.3M ...",San Francisco,CA,https://vcnewsdaily.com/Sanity.io/venture-fund...,"Sanity.io, the developer-focused content platf..."
4,Popmenu,Popmenu helps restaurants take real-time contr...,Series B,17000000,16 Oct 2020,Popmenu Raises $17M Series B,https://vcnewsdaily.com/popmenu/venture-capita...,"ATLANTA, GA, Popmenu has raised a $17 million ...",Atlanta,GA,https://vcnewsdaily.com/Popmenu/venture-fundin...,"Popmenu, an all-in-one digital marketing and o..."
5,QCODE,QCODE is a new podcast and media studio launch...,Series A,6400000,15 Oct 2020,QCODE Pulls In $6.4M Series A Funding,https://vcnewsdaily.com/qcode/venture-capital-...,"LOS ANGELES, CA, QCODE has raised $6.4 million...",Los Angeles,CA,https://vcnewsdaily.com/QCODE/venture-funding.php,"QCODE, a premium content studio and podcast ne..."
6,Sonrai Security,Sonrai Security delivers an enterprise identit...,Series B,20000000,15 Oct 2020,Sonrai Security Nabs $20M Series B Round,https://vcnewsdaily.com/sonrai-security/ventur...,"NEW YORK, NY, Sonrai Security, a leader in pub...",New York,NY,https://vcnewsdaily.com/Sonrai%20Security/vent...,The round was led by Menlo Ventures with full ...
7,Instylla,Instylla is a privately held company focused o...,Series B,25400000,15 Oct 2020,Instylla Scores $25.4M Series B Financing,https://vcnewsdaily.com/instylla/venture-capit...,"BEDFORD, MA, Instylla has announced a $25.4 mi...",Bedford,MA,https://vcnewsdaily.com/Instylla/venture-fundi...,"Instylla, a privately held company developing ..."
8,Menlo Micro,Menlo Micro is reimagining one of the most fun...,Series B,44000000,15 Oct 2020,Menlo Micro Announces $44M Series B,https://vcnewsdaily.com/menlo-micro/venture-ca...,"IRVINE, CA, Menlo Micro, the company responsib...",Irvine,CA,https://vcnewsdaily.com/Menlo%20Micro/venture-...,Menlo Microsystems (Menlo Micro) has created a...
9,Finexio,Finexio is the smart B2B payment network. The...,Growth,23000000,15 Oct 2020,Finexio Closes $23M Financing Round,https://vcnewsdaily.com/finexio/venture-capita...,"ORLANDO, FL, Finexio, a rapidly growing AP pay...",Orlando,FL,https://vcnewsdaily.com/Finexio/venture-fundin...,The round was led with a strategic investment ...


In [41]:
# total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].replace(0,'Undisclosed')
total_funding_data.shape

(817, 12)

In [42]:
# For removing commas and $ from string, to help convert into int
if website_article_links_length > 0:
    total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].str.strip('$')
    total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].replace(',','',regex=True)

# Fill in missing values in 'latest funding amount' with NaNs
# total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].replace('undisclosed',np.NaN)
# total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].replace(np.NaN,'undisclosed')

# Fill in missing 'latest funding round' with 'misc'
total_funding_data['Latest Funding Round'] = total_funding_data['Latest Funding Round'].replace(' ', 'Misc')
total_funding_data['Latest Funding Round'] = total_funding_data['Latest Funding Round'].replace(np.nan, 'Misc')
total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].replace({0, np.nan})
total_funding_data['Latest Funding Amount'] = total_funding_data['Latest Funding Amount'].replace({np.nan, None})

# Convert 'Latest Funding Amount' to int / upload this colum via SQL (lower)

In [43]:
total_funding_data

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Ateios,Ateios is developing a new process to reshape ...,Seed,1250000,17 Oct 2020,Ateios Raises $1.25M in Seed Funding,https://vcnewsdaily.com/ateios/venture-capital...,"NEWBERRY, IN, Ateios announced closing an over...",Newberry,IN,https://vcnewsdaily.com/Ateios/venture-funding...,Ateios (pronounced 'Ah-TAY-ohs') has developed...
1,Vineti,Vineti is the first commercial cloud-based pla...,Series C Extension,33000000,17 Oct 2020,Vineti Grabs $33M Series C Extension,https://vcnewsdaily.com/vineti/venture-capital...,"SAN FRANCISCO, CA, Vineti announced the closin...",San Francisco,CA,https://vcnewsdaily.com/Vineti/venture-funding...,"Vineti, a leading digital platform of record f..."
2,Dren Bio,Dren Bio is a biotechnology company developing...,Series A,60000000,16 Oct 2020,Dren Bio Announces a $60M Series A,https://vcnewsdaily.com/dren-bio/venture-capit...,"SAN CARLOS, CA, Dren Bio announced the closing...",San Carlos,CA,https://vcnewsdaily.com/Dren%20Bio/venture-fun...,"Dren Bio, Inc., a company developing powerful ..."
3,Sanity.io,Sanity is a content platform that helps people...,Series A,9300000,16 Oct 2020,Sanity Announces $9.3M in Series A,https://vcnewsdaily.com/sanity.io/venture-capi...,"SAN FRANCISCO, CA, Sanity.io has raised $9.3M ...",San Francisco,CA,https://vcnewsdaily.com/Sanity.io/venture-fund...,"Sanity.io, the developer-focused content platf..."
4,Popmenu,Popmenu helps restaurants take real-time contr...,Series B,17000000,16 Oct 2020,Popmenu Raises $17M Series B,https://vcnewsdaily.com/popmenu/venture-capita...,"ATLANTA, GA, Popmenu has raised a $17 million ...",Atlanta,GA,https://vcnewsdaily.com/Popmenu/venture-fundin...,"Popmenu, an all-in-one digital marketing and o..."
...,...,...,...,...,...,...,...,...,...,...,...,...
812,Guilded,Guilded is the ultimate chat platform for gami...,Series A,7000000,30 Apr 2020,Guilded Pulls In $7M Series A Financing,https://vcnewsdaily.com/guilded/venture-capita...,"SAN FRANCISCO, CA, Guilded has raised $7 milli...",San Francisco,CA,https://vcnewsdaily.com/Guilded/venture-fundin...,"The funding was led by Matrix Partners, with p..."
813,Stash,Stash is the fast-growing investment app for m...,Series F,112000000,29 Apr 2020,Stash Raises $112M in Series F Funding,https://vcnewsdaily.com/stash/venture-capital-...,"NEW YORK, NY, Stash has raised a $112 million ...",New York,NY,https://vcnewsdaily.com/Stash/venture-funding.php,Stash has raised a $112 million Series F round...
814,Fat Snax,"Fat Snax has been providing low-carb, yet indu...",Series A,4500000,29 Apr 2020,Fat Snax Closes $4.5M Series A,https://vcnewsdaily.com/fat-snax/venture-capit...,"BROOKLYN, NY, Fat Snax has closed a $4.5M Seri...",Brooklyn,NY,https://vcnewsdaily.com/Fat%20Snax/venture-fun...,"Fat Snax, the leading low-carb & keto snack br..."
815,Wise,"Founded in December 2018, Wise, a Techstars NY...",Seed,5700000,29 Apr 2020,Wise Raises $5.7M Seed Round,https://vcnewsdaily.com/wise/venture-capital-f...,"SAN MATEO, CA, FinTech startup that offers ban...",San Mateo,CA,https://vcnewsdaily.com/Wise/venture-funding.php,"Wise, a FinTech startup that offers banking an..."


In [44]:
# Need to figure out what to do with 'undisclosed' -- avg to 'Latest Funding Round'?

In [45]:
total_funding_data.to_csv('VC_News_Daily_Clean.csv',index=False)

In [46]:
pd.read_csv('VC_News_Daily_Clean.csv').head(15)

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Ateios,Ateios is developing a new process to reshape ...,Seed,1250000,17 Oct 2020,Ateios Raises $1.25M in Seed Funding,https://vcnewsdaily.com/ateios/venture-capital...,"NEWBERRY, IN, Ateios announced closing an over...",Newberry,IN,https://vcnewsdaily.com/Ateios/venture-funding...,Ateios (pronounced 'Ah-TAY-ohs') has developed...
1,Vineti,Vineti is the first commercial cloud-based pla...,Series C Extension,33000000,17 Oct 2020,Vineti Grabs $33M Series C Extension,https://vcnewsdaily.com/vineti/venture-capital...,"SAN FRANCISCO, CA, Vineti announced the closin...",San Francisco,CA,https://vcnewsdaily.com/Vineti/venture-funding...,"Vineti, a leading digital platform of record f..."
2,Dren Bio,Dren Bio is a biotechnology company developing...,Series A,60000000,16 Oct 2020,Dren Bio Announces a $60M Series A,https://vcnewsdaily.com/dren-bio/venture-capit...,"SAN CARLOS, CA, Dren Bio announced the closing...",San Carlos,CA,https://vcnewsdaily.com/Dren%20Bio/venture-fun...,"Dren Bio, Inc., a company developing powerful ..."
3,Sanity.io,Sanity is a content platform that helps people...,Series A,9300000,16 Oct 2020,Sanity Announces $9.3M in Series A,https://vcnewsdaily.com/sanity.io/venture-capi...,"SAN FRANCISCO, CA, Sanity.io has raised $9.3M ...",San Francisco,CA,https://vcnewsdaily.com/Sanity.io/venture-fund...,"Sanity.io, the developer-focused content platf..."
4,Popmenu,Popmenu helps restaurants take real-time contr...,Series B,17000000,16 Oct 2020,Popmenu Raises $17M Series B,https://vcnewsdaily.com/popmenu/venture-capita...,"ATLANTA, GA, Popmenu has raised a $17 million ...",Atlanta,GA,https://vcnewsdaily.com/Popmenu/venture-fundin...,"Popmenu, an all-in-one digital marketing and o..."
5,QCODE,QCODE is a new podcast and media studio launch...,Series A,6400000,15 Oct 2020,QCODE Pulls In $6.4M Series A Funding,https://vcnewsdaily.com/qcode/venture-capital-...,"LOS ANGELES, CA, QCODE has raised $6.4 million...",Los Angeles,CA,https://vcnewsdaily.com/QCODE/venture-funding.php,"QCODE, a premium content studio and podcast ne..."
6,Sonrai Security,Sonrai Security delivers an enterprise identit...,Series B,20000000,15 Oct 2020,Sonrai Security Nabs $20M Series B Round,https://vcnewsdaily.com/sonrai-security/ventur...,"NEW YORK, NY, Sonrai Security, a leader in pub...",New York,NY,https://vcnewsdaily.com/Sonrai%20Security/vent...,The round was led by Menlo Ventures with full ...
7,Instylla,Instylla is a privately held company focused o...,Series B,25400000,15 Oct 2020,Instylla Scores $25.4M Series B Financing,https://vcnewsdaily.com/instylla/venture-capit...,"BEDFORD, MA, Instylla has announced a $25.4 mi...",Bedford,MA,https://vcnewsdaily.com/Instylla/venture-fundi...,"Instylla, a privately held company developing ..."
8,Menlo Micro,Menlo Micro is reimagining one of the most fun...,Series B,44000000,15 Oct 2020,Menlo Micro Announces $44M Series B,https://vcnewsdaily.com/menlo-micro/venture-ca...,"IRVINE, CA, Menlo Micro, the company responsib...",Irvine,CA,https://vcnewsdaily.com/Menlo%20Micro/venture-...,Menlo Microsystems (Menlo Micro) has created a...
9,Finexio,Finexio is the smart B2B payment network. The...,Growth,23000000,15 Oct 2020,Finexio Closes $23M Financing Round,https://vcnewsdaily.com/finexio/venture-capita...,"ORLANDO, FL, Finexio, a rapidly growing AP pay...",Orlando,FL,https://vcnewsdaily.com/Finexio/venture-fundin...,The round was led with a strategic investment ...


In [47]:
# Create excel file in order to upload into Microsoft SQL in order to upload dataset
total_funding_data.to_excel('VC_News_Daily_Clean.xlsx')

In [48]:
# Sensitive information withheld - via NULL
driver_name = '{ODBC Driver 17 for SQL Server}'
server_name = 'DESKTOP-OVRCKME'
database_name = 'VCNewsDailyDB'
trusted_connection = 'yes'

In [49]:
# Connect to SQL Server
conn = pyodbc.connect(Driver=driver_name,
                      Server=server_name,
                      Database=database_name,
                      Trusted_Connection=trusted_connection)
cursor = conn.cursor()

In [50]:
# Before adding new information - SQL Table as is, without the reordering
sql_raw_total_funding_data = pd.read_sql('SELECT * FROM VC_News_Daily',conn)
sql_raw_total_funding_data.head(15)

,index,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,0,Sanity.io,Sanity is a content platform that helps people...,Series A,9300000,16 Oct 2020,Sanity Announces $9.3M in Series A,https://vcnewsdaily.com/sanity.io/venture-capi...,"SAN FRANCISCO, CA, Sanity.io has raised $9.3M ...",San Francisco,CA,https://vcnewsdaily.com/Sanity.io/venture-fund...,"Sanity.io, the developer-focused content platf..."
1,1,Popmenu,Popmenu helps restaurants take real-time contr...,Series B,17000000,16 Oct 2020,Popmenu Raises $17M Series B,https://vcnewsdaily.com/popmenu/venture-capita...,"ATLANTA, GA, Popmenu has raised a $17 million ...",Atlanta,GA,https://vcnewsdaily.com/Popmenu/venture-fundin...,"Popmenu, an all-in-one digital marketing and o..."
2,2,QCODE,QCODE is a new podcast and media studio launch...,Series A,6400000,15 Oct 2020,QCODE Pulls In $6.4M Series A Funding,https://vcnewsdaily.com/qcode/venture-capital-...,"LOS ANGELES, CA, QCODE has raised $6.4 million...",Los Angeles,CA,https://vcnewsdaily.com/QCODE/venture-funding.php,"QCODE, a premium content studio and podcast ne..."
3,3,Sonrai Security,Sonrai Security delivers an enterprise identit...,Series B,20000000,15 Oct 2020,Sonrai Security Nabs $20M Series B Round,https://vcnewsdaily.com/sonrai-security/ventur...,"NEW YORK, NY, Sonrai Security, a leader in pub...",New York,NY,https://vcnewsdaily.com/Sonrai%20Security/vent...,The round was led by Menlo Ventures with full ...
4,4,Instylla,Instylla is a privately held company focused o...,Series B,25400000,15 Oct 2020,Instylla Scores $25.4M Series B Financing,https://vcnewsdaily.com/instylla/venture-capit...,"BEDFORD, MA, Instylla has announced a $25.4 mi...",Bedford,MA,https://vcnewsdaily.com/Instylla/venture-fundi...,"Instylla, a privately held company developing ..."
5,5,Menlo Micro,Menlo Micro is reimagining one of the most fun...,Series B,44000000,15 Oct 2020,Menlo Micro Announces $44M Series B,https://vcnewsdaily.com/menlo-micro/venture-ca...,"IRVINE, CA, Menlo Micro, the company responsib...",Irvine,CA,https://vcnewsdaily.com/Menlo%20Micro/venture-...,Menlo Microsystems (Menlo Micro) has created a...
6,14,M1 Finance,M1 helps people manage and grow their money in...,Series C,45000000,15 Oct 2020,M1 Finance Closes $45M Series C Round,https://vcnewsdaily.com/m1-finance/venture-cap...,"CHICAGO, IL, M1 Finance announced a $45 millio...",Chicago,IL,https://vcnewsdaily.com/M1%20Finance/venture-f...,"M1 Finance, the Finance Super App that offers ..."
7,6,Finexio,Finexio is the smart B2B payment network. The...,Growth,23000000,15 Oct 2020,Finexio Closes $23M Financing Round,https://vcnewsdaily.com/finexio/venture-capita...,"ORLANDO, FL, Finexio, a rapidly growing AP pay...",Orlando,FL,https://vcnewsdaily.com/Finexio/venture-fundin...,The round was led with a strategic investment ...
8,7,Acceldata,"Founded in 2018, Acceldata is the industry's f...",Series A,8500000,15 Oct 2020,Acceldata Raises $8.5M Series A Round,https://vcnewsdaily.com/acceldata/venture-capi...,"PALO ALTO, CA, Acceldata, the world's first en...",Palo Alto,CA,https://vcnewsdaily.com/Acceldata/venture-fund...,The round was led by Sorenson Ventures with pa...
9,15,Polygon.io,"Polygon.io, a leading market data platform tha...",Series A,6000000,14 Oct 2020,Polygon.io Shapes Up with $6M Series A Round,https://vcnewsdaily.com/polygon.io/venture-cap...,"ATLANTA, GA, Polygon.io has raised a $6M Serie...",Atlanta,GA,https://vcnewsdaily.com/Polygon.io/venture-fun...,"Polygon.io, a leading financial data platform,..."


In [51]:
# Sensitive information withheld
engine = sqlalchemy.create_engine('mssql+pyodbc://@DESKTOP-OVRCKME/VCNewsDailyDB?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

In [52]:
total_funding_data.dtypes

Company                  object
Company Description      object
Latest Funding Round     object
Latest Funding Amount    object
Funding Date             object
Funding Title            object
Website Article Link     object
Funding Headline         object
City                     object
State                    object
Website Company Link     object
Funding Article          object
dtype: object

total_funding_data[['Latest Funding Amount']].astype('int')

In [53]:
total_funding_data.dtypes

Company                  object
Company Description      object
Latest Funding Round     object
Latest Funding Amount    object
Funding Date             object
Funding Title            object
Website Article Link     object
Funding Headline         object
City                     object
State                    object
Website Company Link     object
Funding Article          object
dtype: object

In [54]:
# Upload 'Latest Funding Amount' as int, in order to process calculations

total_funding_data.to_sql('VC_News_Daily',con=engine,if_exists='replace')

In [55]:
sql_total_funding_data = pd.read_sql('SELECT * FROM VC_News_Daily',conn).sort_values('index')
sql_total_funding_data.head(15)

,index,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,0,Ateios,Ateios is developing a new process to reshape ...,Seed,1250000,17 Oct 2020,Ateios Raises $1.25M in Seed Funding,https://vcnewsdaily.com/ateios/venture-capital...,"NEWBERRY, IN, Ateios announced closing an over...",Newberry,IN,https://vcnewsdaily.com/Ateios/venture-funding...,Ateios (pronounced 'Ah-TAY-ohs') has developed...
1,1,Vineti,Vineti is the first commercial cloud-based pla...,Series C Extension,33000000,17 Oct 2020,Vineti Grabs $33M Series C Extension,https://vcnewsdaily.com/vineti/venture-capital...,"SAN FRANCISCO, CA, Vineti announced the closin...",San Francisco,CA,https://vcnewsdaily.com/Vineti/venture-funding...,"Vineti, a leading digital platform of record f..."
3,2,Dren Bio,Dren Bio is a biotechnology company developing...,Series A,60000000,16 Oct 2020,Dren Bio Announces a $60M Series A,https://vcnewsdaily.com/dren-bio/venture-capit...,"SAN CARLOS, CA, Dren Bio announced the closing...",San Carlos,CA,https://vcnewsdaily.com/Dren%20Bio/venture-fun...,"Dren Bio, Inc., a company developing powerful ..."
4,3,Sanity.io,Sanity is a content platform that helps people...,Series A,9300000,16 Oct 2020,Sanity Announces $9.3M in Series A,https://vcnewsdaily.com/sanity.io/venture-capi...,"SAN FRANCISCO, CA, Sanity.io has raised $9.3M ...",San Francisco,CA,https://vcnewsdaily.com/Sanity.io/venture-fund...,"Sanity.io, the developer-focused content platf..."
5,4,Popmenu,Popmenu helps restaurants take real-time contr...,Series B,17000000,16 Oct 2020,Popmenu Raises $17M Series B,https://vcnewsdaily.com/popmenu/venture-capita...,"ATLANTA, GA, Popmenu has raised a $17 million ...",Atlanta,GA,https://vcnewsdaily.com/Popmenu/venture-fundin...,"Popmenu, an all-in-one digital marketing and o..."
7,5,QCODE,QCODE is a new podcast and media studio launch...,Series A,6400000,15 Oct 2020,QCODE Pulls In $6.4M Series A Funding,https://vcnewsdaily.com/qcode/venture-capital-...,"LOS ANGELES, CA, QCODE has raised $6.4 million...",Los Angeles,CA,https://vcnewsdaily.com/QCODE/venture-funding.php,"QCODE, a premium content studio and podcast ne..."
8,6,Sonrai Security,Sonrai Security delivers an enterprise identit...,Series B,20000000,15 Oct 2020,Sonrai Security Nabs $20M Series B Round,https://vcnewsdaily.com/sonrai-security/ventur...,"NEW YORK, NY, Sonrai Security, a leader in pub...",New York,NY,https://vcnewsdaily.com/Sonrai%20Security/vent...,The round was led by Menlo Ventures with full ...
11,7,Instylla,Instylla is a privately held company focused o...,Series B,25400000,15 Oct 2020,Instylla Scores $25.4M Series B Financing,https://vcnewsdaily.com/instylla/venture-capit...,"BEDFORD, MA, Instylla has announced a $25.4 mi...",Bedford,MA,https://vcnewsdaily.com/Instylla/venture-fundi...,"Instylla, a privately held company developing ..."
12,8,Menlo Micro,Menlo Micro is reimagining one of the most fun...,Series B,44000000,15 Oct 2020,Menlo Micro Announces $44M Series B,https://vcnewsdaily.com/menlo-micro/venture-ca...,"IRVINE, CA, Menlo Micro, the company responsib...",Irvine,CA,https://vcnewsdaily.com/Menlo%20Micro/venture-...,Menlo Microsystems (Menlo Micro) has created a...
13,9,Finexio,Finexio is the smart B2B payment network. The...,Growth,23000000,15 Oct 2020,Finexio Closes $23M Financing Round,https://vcnewsdaily.com/finexio/venture-capita...,"ORLANDO, FL, Finexio, a rapidly growing AP pay...",Orlando,FL,https://vcnewsdaily.com/Finexio/venture-fundin...,The round was led with a strategic investment ...


In [56]:
sql_total_funding_data.shape

(817, 13)